# Penguins in Production

This notebook creates a [SageMaker Pipeline](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_building_pipeline.html) to build an end-to-end Machine Learning system to solve the problem of classifying penguin species. With a SageMaker Pipeline, you can create, automate, and manage end-to-end Machine Learning workflows at scale.

You can find more information about Amazon SageMaker in the [Amazon SageMaker Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html). The [AWS Machine Learning Blog](https://aws.amazon.com/blogs/machine-learning/) is an excellent source to stay up-to-date with SageMaker.

This example uses the [Penguins dataset](https://www.kaggle.com/parulpandey/palmer-archipelago-antarctica-penguin-data), the [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html) library, and the [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/). 

<img src='https://imgur.com/orZWHly.png' alt='Penguins dataset' width="800">

## Table of Contents

1. [Session 1 - Production Machine Learning is Different](#Session-1---Production-Machine-Learning-is-Different) 
2. [Session 2 - Implementing a Production Pipeline](#Session-2---Implementing-a-Production-Pipeline)
3. [Session 3 - Building a Model](#Session-3---Building-a-Model)
4. [Session 4 - Evaluating and Registering The Model](#Session-4---Evaluating-and-Registering-The-Model)
5. [Session 5 - Deploying The Model](#Session-5---Deploying-The-Model)
6. [Session 6 - Monitoring](#Session-6---Monitoring)
7. [Running the Pipeline](#Running-the-Pipeline)

This notebook is part of the [Machine Learning School](https://www.ml.school) program.

<div class="alert" style="background-color:#6e420c; color: #fff"><strong>Note:</strong> 
    Make sure you run the <strong><a href="penguins-setup.ipynb" style="color: #0397a7">Setup notebook</a></strong> once at the start of the program and before running this notebook. After doing that, ensure you are using a "TensorFlow 2.6 Python 3.8 CPU Optimized" image and the "packages" start-up script to run this notebook.
</div>

In [23]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(parents=True, exist_ok=True)

sys.path.append(f"./{CODE_FOLDER}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Session 1 - Production Machine Learning is Different

In this session we'll run Exploratory Data Analysis on the [Penguins dataset](https://www.kaggle.com/parulpandey/palmer-archipelago-antarctica-penguin-data). We'll split and transform the data using a [Scikit-Learn Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

In [24]:
import boto3
import logging
import numpy as np
import tensorflow as tf
import urllib.request
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import sagemaker
import sklearn

from plotly.subplots import make_subplots
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sagemaker.s3 import S3Uploader

# By default, The SageMaker SDK logs events related to the default
# configuration using the INFO level. To prevent these from spoiling
# the output of this notebook cells, we can change the logging
# level to ERROR instead.
logging.getLogger("sagemaker.config").setLevel(logging.ERROR)

## Step 1 - Upload Dataset to S3

Let's create the S3 bucket where we will organize every resource we are going to use during the program.

<div class="alert" style="background-color:#6e420c; color: #fff"><strong>Note:</strong> 
    You need to select a unique name for the <strong>BUCKET</strong> constant. If you want to create a bucket in a region other than <strong>us-east-1</strong>, you need to use the "--create-bucket-configuration" argument when creating the bucket. You can see an example below.
</div>

Example of how to specify a region different from `us-east-1` when creating a bucket:

```
!aws s3api create-bucket --bucket $BUCKET --create-bucket-configuration LocationConstraint="eu-west-1"
```

The `LocationConstraint` argument should specify the region where you want to create the bucket. The example above creates the bucket in the `eu-west-1` region.

In [25]:
BUCKET = "mlschool"

!aws s3api create-bucket --bucket $BUCKET

{
    "Location": "/mlschool"
}


After we have a bucket, we can download the [Penguins dataset](https://www.kaggle.com/parulpandey/palmer-archipelago-antarctica-penguin-data) and store it in the bucket.

In [26]:
S3_LOCATION = f"s3://{BUCKET}/penguins"
DATA_FILEPATH = Path().resolve() / "data.csv"

urllib.request.urlretrieve(
    "https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins_size.csv", 
    DATA_FILEPATH
)

S3Uploader.upload(local_path=str(DATA_FILEPATH), desired_s3_uri=S3_LOCATION)

's3://mlschool/penguins/data.csv'

## Step 2 - Analyzing the Dataset

Let's load the Penguins dataset. 

The dataset contains the following columns:

1. `species`: The species of a penguin. This is the column we want to predict.
2. `island`: The island where the penguin was found
3. `culmen_length_mm`: The length of the penguin's culmen (bill) in millimeters
4. `culmen_depth_mm`: The depth of the penguin's culmen in millimeters
5. `flipper_length_mm`: The length of the penguin's flipper in millimeters
6. `body_mass_g`: The body mass of the penguin in grams
7. `sex`: The sex of the penguin

In [28]:
# penguins = pd.read_csv(DATA_FILEPATH / "data.csv")
# penguins.head()

Now, let's get the summary statistics for the features in our dataset.

In [29]:
# penguins.describe(include='all')

NameError: name 'penguins' is not defined

The distribution of the categories in our dataset are:

- `species`: There are 3 species of penguins in the dataset: Adelie (152), Gentoo (124), and Chinstrap (68).
- `island`: Penguins are from 3 islands: Biscoe (168), Dream (124), and Torgersen (52).
- `sex`: We have 168 male penguins, 165 female penguins, and 1 penguin with an ambiguous gender ('.').

In [30]:
# species_distribution = penguins['species'].value_counts()
# island_distribution = penguins['island'].value_counts()
# sex_distribution = penguins['sex'].value_counts()

# print(species_distribution)
# print()
# print(island_distribution)
# print()
# print(sex_distribution)

NameError: name 'penguins' is not defined

Let's replace the ambiguous value in the `sex` column with a null value.

## Step 3 - Building a Scikit-Learn Pipeline

Let's create a Scikit-Learn pipeline to transform the dataset. Here are the transformations that will happen when we use this pipeline:

* Numeric columns: First, we will impute any missing values using the mean of the column. Then, we will scale the values using a standard scaler.
* Categorical columns: First, we will impute any missing values using the value that shows up more frequently in the data. Then, we will one-hot encode the column.

In [31]:
# numeric_features = penguins.select_dtypes(include=["float64"]).columns.tolist()
# numeric_transformer = sklearn.pipeline.Pipeline(
#     steps=[
#         ("imputer", SimpleImputer(strategy="mean")),
#         ("scaler", StandardScaler()),
#     ]
# )

# categorical_transformer = sklearn.pipeline.Pipeline(
#     steps=[
#         ("imputer", SimpleImputer(strategy="most_frequent")),
#         ("encoder", OneHotEncoder()),
#     ]
# )

# target_transformer = sklearn.pipeline.Pipeline(
#     steps=[
#         # ("imputer", SimpleImputer(strategy="most_frequent")),
#         ("encoder", LabelEncoder()),
#     ]
# )

# preprocessor = ColumnTransformer(
#     transformers=[
#         # ("t", target_transformer, ["species"]),
#         ("numeric", numeric_transformer, numeric_features),
#         ("categorical", categorical_transformer, ["island"]),        
#     ]
# )

# pipeline = sklearn.pipeline.Pipeline(
#     steps=[
#         ("preprocessing", preprocessor)
#     ]
# )

In [32]:
# penguins.info()

We can display a diagram of our pipeline.

In [33]:
# set_config(display="diagram")
# pipeline

## Step 4 - Splitting and Transforming the Dataset

Now it's time to split and transform the dataset.

Before we split the dataset, let's remove the `sex` column because we don't need it to create a model.

## Questions

Answering these questions will help you understand the material we discussed during this session. Notice that each question could have one or more correct answers.


<div style="margin: 30px 0 10px 0;"><span style="font-size: 1.1em; padding:4px; background-color: #b8bf9f; color: #000;"><strong>Question 1.1</strong></span></div>

What will happen if we apply the SciKit-Learn transformation pipeline to the entire dataset before splitting it?

1. Scaling will use the global statistics of the dataset, leaking the mean and variance of the test samples into the training process.
2. Imputing the missing numeric values will use the global mean, leading to data leakage.
3. The transformation pipeline expects multiple sets so it wouldn't work.
4. We will reduce the number of lines of code we need to transform the dataset.


<div style="margin: 30px 0 10px 0;"><span style="font-size: 1.1em; padding:4px; background-color: #b8bf9f; color: #000;"><strong>Question 1.2</strong></span></div>

A hospital wants to predict which patients are prone to get a disease based on their medical history. They use weak supervision to automatically label the data using a set of heuristics. What are some of the disadvantages of weak supervision?

1. Weak supervision doesn't scale to large datasets.
2. Weak supervision doesn't adapt well to any changes that require relabeling.
3. Weak supervision produces noisy labels.
4. We might not be able to use weak supervision to label every data sample.


<div style="margin: 30px 0 10px 0;"><span style="font-size: 1.1em; padding:4px; background-color: #b8bf9f; color: #000;"><strong>Question 1.3</strong></span></div>

When collecting the information about the penguins, the scientists encountered a few rare species. To prevent these samples from not showing when splitting the data, they recommended using Stratified Sampling. Which of the following statements about Stratified Sampling are correct?

1. Stratified Sampling assigns every sample of the population an equal chance of being selected.
2. Stratified Sampling preserves the original distribution of different groups in the data.
3. Stratified Sampling requires having a larger dataset compared to Random Sampling.
4. Stratified Sampling can't be used when is not possible to divide all samples into groups.


<div style="margin: 30px 0 10px 0;"><span style="font-size: 1.1em; padding:4px; background-color: #b8bf9f; color: #000;"><strong>Question 1.4</strong></span></div>

Using more features to build a model will not necessarily lead to better predictions. Which of the following are drawbacks from adding more features?

1. More features in a dataset increases the opportunity for data leakage.
2. More features in a dataset increases the opportunity for overfitting.
3. More features in a dataset increases the memory necessary to serve a model.
4. More features in a dataset increases the development and maintenance time of a model. 


<div style="margin: 30px 0 10px 0;"><span style="font-size: 1.1em; padding:4px; background-color: #b8bf9f; color: #000;"><strong>Question 1.5</strong></span></div>

A bank wants to store every transaction they handle in a set of files in the cloud. Each file will contain the transactions generated in a day. The team who will manage these files wants to optimize the storage space and downloding speed. What format should the bank use to store the transactions?

1. The bank should store the data in JSON format.
2. The bank should store the data in CSV format.
3. The bank should store the data in Parquet format.
4. The bank should store the data in Pandas format.


## Assignments


* <span style="padding:4px; background-color: #f2a68a; color: #000;"><strong>Assignment 1.1</strong></span> Familiarize yourself with the dataset we used to kick start the project. Extend the Exploratory Data Analysis in this notebook with any other analysis that you consider interesting.

* <span style="padding:4px; background-color: #f2a68a; color: #000;"><strong>Assignment 1.2</strong></span> The code in this session removes the `sex` column from the dataset before transforming the data. Modify the Scikit-Learn Pipeline to include an additional step that removes the `sex` column.

* <span style="padding:4px; background-color: #f2a68a; color: #000;"><strong>Assignment 1.3</strong></span> The code in this session uses Random Sampling to split the dataset. Modify the code to use Stratified Sampling instead.

* <span style="padding:4px; background-color: #f2a68a; color: #000;"><strong>Assignment 1.4</strong></span> You can use [Amazon SageMaker Data Wrangler](https://aws.amazon.com/sagemaker/data-wrangler/) to complete each step of the data preparation workflow (including data selection, cleansing, exploration, visualization, and processing at scale) from a single visual interface. For this assignment, load the Data Wrangler interface and use it to build the same transformations we implemented using the Scikit-Learn Pipeline. If you have questions, open the [Penguins Data Flow](penguins.flow) included in this repository.

* <span style="padding:4px; background-color: #f2a68a; color: #000;"><strong>Assignment 1.5</strong></span> You can use [Amazon SageMaker Ground Truth](https://aws.amazon.com/sagemaker/data-labeling/) to label data and generate labeled synthetic data. You can configure the service to use workers from either Amazon Mechanical Turk, a vendor company, or an internal, private workforce along with machine learning to enable you to create a labeled dataset. For this assignment, familiarize yourself with the service and setup a simple "Text Classification (Multi-label)" labeling job.


# Session 2 - Implementing a Production Pipeline

In this session we'll build a simple [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with one step to preprocess the data.

We'll use a [Processing Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing) with a [SKLearnProcessor](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#scikit-learn-processor) to execute a preprocessing script. Check the [SageMaker Pipelines Overview](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) for an introduction to the fundamental components of a SageMaker Pipeline.

In [34]:
import os
import json
import tempfile

from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import CacheConfig
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig
from sagemaker.workflow.pipeline_context import PipelineSession

pipeline_session = PipelineSession(default_bucket=BUCKET)
sagemaker_session = sagemaker.session.Session()

pipeline_definition_config = PipelineDefinitionConfig(use_custom_job_prefix=True)
sagemaker_client = boto3.client("sagemaker")
iam_client = boto3.client("iam")
role = sagemaker.get_execution_role()
region = boto3.Session().region_name


## Step 1 - Creating the Preprocessing Script

We need to create a script to transform and split the dataset. This is the script that we'll run using the Processing Job.

The script will save the Scikit-Learn pipeline that we use to preprocess the data and the list of target classes.

In [35]:
%%writefile {CODE_FOLDER}/preprocessor.py

import os
import numpy as np
import pandas as pd
import argparse
import json
import tarfile

from pathlib import Path
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, OrdinalEncoder
from pickle import dump, load

import time
import sys
from io import StringIO

import shutil

import csv
import joblib


from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from pickle import dump



from sagemaker_containers.beta.framework import (
    content_types, encoders, env, modules, transformer, worker)



TARGET_COLUMN = "species"
feature_columns_names = [
    'island',
    'culmen_length_mm',
    'culmen_depth_mm', 
    'flipper_length_mm',
    'body_mass_g',
    'sex']

feature_columns_dtype = {
    'island': "category",
    'culmen_length_mm': "float64",
    'culmen_depth_mm': "float64",
    'flipper_length_mm': "float64",
    'body_mass_g': "float64",
    'sex': "category"}

label_column_dtype = {'species': "category"}


def input_fn(input_data, content_type):
    """Parse input data payload

    We currently only take csv input. Since we need to process both labelled
    and unlabelled data we first determine whether the label column is present
    by looking at how many columns were provided.
    """
    if content_type == "text/csv":
        # Read the raw input data as CSV.
        df = pd.read_csv(StringIO(input_data), header=None)

        if len(df.columns) == len(feature_columns_names) + 1:
            # This is a labelled example
            df.columns = [TARGET_COLUMN] + feature_columns_names
            
        elif len(df.columns) == len(feature_columns_names):
            # This is an unlabelled example.
            df.columns = feature_columns_names

        return df
    
    else:
        raise ValueError(f"{content_type} is not supported.!")


def output_fn(prediction, accept):
    """Format prediction output

    The default accept/content-type between containers for serial inference is JSON.
    We also want to set the ContentType or mimetype as the same value as accept so the next
    container can read the response payload correctly.
    """
    print("SANTI - output_fn", prediction)
    
    if accept == "application/json":
        instances = []
        for row in prediction.tolist():
            instances.append(row)
        json_output = {"instances": instances}
        
        print("Accept JSON:", json_output)

        return worker.Response(json.dumps(json_output), mimetype=accept)
    elif accept == 'text/csv':
        
        print("Accept CSV:", prediction)
        
        return worker.Response(encoders.encode(prediction, accept), mimetype=accept)
    else:
        raise RuntimeException(f"{accept} accept type is not supported.")


def predict_fn(input_data, model):
    """
    Preprocess input data
    """
    
    features_transformer = model["features_transformer"]
    features = features_transformer.transform(input_data)
    
    if TARGET_COLUMN in input_data:
        target = np.array(input_data[TARGET_COLUMN]).reshape(-1, 1)
        
        target_transformer = model["target_transformer"]
        target = target_transformer.transform(target)
        
        # return np.insert(features, 0, input_data[TARGET_COLUMN], axis=1)
        # return np.insert(features, 0, target, axis=1)
        return np.hstack((target, features))
    
    return features


def model_fn(model_dir):
    """Deserialize fitted model
    """
    
    return {
        "target_transformer": joblib.load(os.path.join(model_dir, "target.joblib")),
        "features_transformer": joblib.load(os.path.join(model_dir, "features.joblib")),
    }


def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z


def preprocess(base_directory):
    input_directory = Path(base_directory) / "input"
    files = [file for file in input_directory.glob("*.csv")]
    
    if len(files) == 0:
        raise ValueError(f"The are no CSV files in {str(input_directory)}/")
        
    raw_data = [
        pd.read_csv(
            file, 
            # header=None,
            # names=[TARGET_COLUMN] + feature_columns_names,
            # dtype=merge_two_dicts(label_column_dtype, feature_columns_dtype)
        ) for file in files
    ]
    df = pd.concat(raw_data)
    
    df = df.sample(frac=1, random_state=42)
    
    
    target_transformer = ColumnTransformer(
        transformers=[("species", OrdinalEncoder(), [0])]
    )
    
    numeric_transformer = make_pipeline(
        SimpleImputer(strategy="mean"),
        StandardScaler()
    )

    categorical_transformer = make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OneHotEncoder()
    )
    
    features_transformer = ColumnTransformer(
        transformers=[
            ("numeric", numeric_transformer, make_column_selector(dtype_exclude="object")),
            ("categorical", categorical_transformer, ["island"]),
        ]
    )
    

    df_train, temp = train_test_split(df, test_size=0.3)
    df_validation, df_test = train_test_split(temp, test_size=0.5)
    
    
    y_train = target_transformer.fit_transform(np.array(df_train.species.values).reshape(-1, 1))
    y_validation = target_transformer.transform(np.array(df_validation.species.values).reshape(-1, 1))
    y_test = target_transformer.transform(np.array(df_test.species.values).reshape(-1, 1))
    
    
    df_train.drop(TARGET_COLUMN, axis=1, inplace=True)
    df_validation.drop(TARGET_COLUMN, axis=1, inplace=True)
    df_test.drop(TARGET_COLUMN, axis=1, inplace=True)

    
    X_train = features_transformer.fit_transform(df_train)
    X_validation = features_transformer.transform(df_validation)
    X_test = features_transformer.transform(df_test)
    
    train = np.concatenate((X_train, y_train), axis=1)
    validation = np.concatenate((X_validation, y_validation), axis=1)
    test = np.concatenate((X_test, y_test), axis=1)

    train_path = Path(base_directory) / "train"
    validation_path = Path(base_directory) / "validation"
    test_path = Path(base_directory) / "test"

    train_path.mkdir(parents=True, exist_ok=True)
    validation_path.mkdir(parents=True, exist_ok=True)
    test_path.mkdir(parents=True, exist_ok=True)

    pd.DataFrame(train).to_csv(train_path / "train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(validation_path / "validation.csv", header=False, index=False)
    pd.DataFrame(test).to_csv(test_path / "test.csv", header=False, index=False)
    
    
    joblib.dump(target_transformer, "target.joblib")
    joblib.dump(features_transformer, "features.joblib")
    
    
    model_path = Path(base_directory) / "model"
    model_path.mkdir(parents=True, exist_ok=True)
    
    
    with tarfile.open(f"{str(model_path / 'model.tar.gz')}", "w:gz") as tar:
        tar.add(f"target.joblib")
        tar.add(f"features.joblib")
    
    
#     df = df.sample(frac=1, random_state=42)

#     df_train, temp = train_test_split(df, test_size=0.3)
#     df_validation, df_test = train_test_split(temp, test_size=0.5)
    
#     target_transformer = ColumnTransformer(
#         transformers=[("species", OrdinalEncoder(), [0])]
#     )
    
#     target_transformer.fit(np.array(df_train["species"].values).reshape(-1, 1))
#     joblib.dump(target_transformer, os.path.join(model_directory, "target.joblib"))
    
#     df_train.drop(TARGET_COLUMN, axis=1, inplace=True)
    
#     numeric_transformer = make_pipeline(
#         SimpleImputer(strategy="mean"),
#         StandardScaler()
#     )

#     categorical_transformer = make_pipeline(
#         SimpleImputer(strategy="most_frequent"),
#         OneHotEncoder()
#     )
    
#     features_transformer = ColumnTransformer(
#         transformers=[
#             ("numeric", numeric_transformer, make_column_selector(dtype_exclude="category")),
#             ("categorical", categorical_transformer, ["island"]),
#         ]
#     )

#     df_train = features_transformer.fit_transform(df_train)
#     df_validation = features_transformer.transform(df_validation)
#     df_test = features_transformer.transform(df_test)
    
#     joblib.dump(features_transformer, os.path.join(model_directory, "features.joblib"))

if __name__ == "__main__":
#     parser = argparse.ArgumentParser()

#     parser.add_argument("--output-data-dir", type=str, default=os.environ["SM_OUTPUT_DATA_DIR"])
#     parser.add_argument("--model-dir", type=str, default=os.environ["SM_MODEL_DIR"])
#     parser.add_argument("--train", type=str, default=os.environ["SM_CHANNEL_TRAIN"])

#     args = parser.parse_args()
    
#     fit_transformation_pipelines(
#         input_directory=Path(args.train), 
#         model_directory=args.model_dir
#     )
    preprocess(base_directory="/opt/ml/processing")

Overwriting code/preprocessor.py


In [ ]:
import pandas as pd
import joblib
from preprocessor import preprocess

with tempfile.TemporaryDirectory() as directory: 
    data = pd.read_csv(DATA_FILEPATH)
    
    input_directory = Path(directory) / "input"
    input_directory.mkdir(parents=True, exist_ok=True)
    
    data.to_csv(input_directory / "data.csv")
    
    preprocess(
        base_directory=Path(directory)
    )
    
    print(f"output: {os.listdir(directory)}")
    
    df = pd.read_csv(Path(directory) / "train" / "train.csv", header=None)
    
    print(df.head(10))
#     t = joblib.load(os.path.join(directory, "target.joblib"))
    
#     x = t.transform([["Adelie"]])
#     print(x)

In [ ]:
features = [[1, 2, 3], [4, 5, 6]]

target = [[10], [20]]


# np.insert(features, 0, target, axis=1)

np.hstack((target, features))


In [36]:
%%writefile {CODE_FOLDER}/split.py

import os
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from pickle import dump


# This is the location where the SageMaker Processing job
# will save the input dataset.
BASE_DIRECTORY = "/opt/ml/processing"
DATA_FILEPATH = Path(BASE_DIRECTORY) / "input" / "data.csv"


def split(base_directory, data_filepath):
    """
    TBD
    """

    df = pd.read_csv(data_filepath)
    df = df.sample(frac=1, random_state=42)

    df_train, temp = train_test_split(df, test_size=0.3)
    df_validation, df_test = train_test_split(temp, test_size=0.5)
    
    train_path = Path(base_directory) / "train"
    validation_path = Path(base_directory) / "validation"
    test_path = Path(base_directory) / "test"

    df_train.to_csv(train_path / "train.csv", header=False, index=False)
    df_validation.to_csv(validation_path / "validation.csv", header=False, index=False)
    df_test.to_csv(test_path / "test.csv", header=False, index=False)


if __name__ == "__main__":
    split(BASE_DIRECTORY, DATA_FILEPATH)

Overwriting code/split.py


In [ ]:
# from sagemaker.sklearn.estimator import SKLearn

# FRAMEWORK_VERSION = "1.2-1"

# sklearn_preprocessor = SKLearn(
#     entry_point=f"{CODE_FOLDER}/preprocessor.py",
#     role=role,
#     framework_version=FRAMEWORK_VERSION,
#     instance_type="ml.m5.xlarge",
#     sagemaker_session=sagemaker_session,
# )

In [ ]:
# sklearn_preprocessor.fit({"train": "s3://mlschool/penguins/data"})

In [ ]:
# transformer = sklearn_preprocessor.transformer(
#     instance_count=1, 
#     instance_type="ml.m5.xlarge", 
#     assemble_with="Line", 
#     accept="text/csv"
# )


# transformer.transform("s3://mlschool/penguins/data", content_type="text/csv")
# print("Waiting for transform job: " + transformer.latest_transform_job.job_name)

# transformer.wait()
# preprocessed_train = transformer.output_path
# preprocessed_train


## HERE

In [37]:
from sagemaker.tuner import HyperparameterTuner
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TuningStep
from sagemaker.parameter import IntegerParameter
from sagemaker.inputs import TrainingInput
from sagemaker.tensorflow import TensorFlow
from sagemaker.workflow.steps import TrainingStep
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.tensorflow import TensorFlowProcessor
from sagemaker.model import Model
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.predictor import Predictor
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.functions import Join
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.parameters import ParameterFloat
from sagemaker.inputs import CreateModelInput, TransformInput
from sagemaker.transformer import Transformer
from sagemaker.workflow.steps import CreateModelStep, TransformStep


cache_config = CacheConfig(
    enable_caching=True, 
    expire_after="15d"
)

In [38]:
preprocessor_processor = SKLearnProcessor(
    base_job_name="split-processor",
    framework_version="1.2-1",
    instance_type="ml.m5.xlarge",
    instance_count=1,
    role=role,
    sagemaker_session=pipeline_session
)

preprocess_step = ProcessingStep(
    name="preprocess-data",
    step_args=preprocessor_processor.run(
        code=f"{CODE_FOLDER}/preprocessor.py",
        inputs=[
            ProcessingInput(source="s3://mlschool/penguins/data", destination="/opt/ml/processing/input"),  
        ],
        outputs=[
            ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
            ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
            ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
            ProcessingOutput(output_name="model", source="/opt/ml/processing/model"),
        ]
    ),
    cache_config=cache_config
)

# estimator1 = SKLearn(
#     entry_point=f"{CODE_FOLDER}/preprocessor.py",
#     role=role,
#     framework_version="1.2-1",
#     instance_type="ml.m5.xlarge",
#     sagemaker_session=pipeline_session,
# )


# preprocessor_step = TrainingStep(
#     name="fit-transformer",
#     step_args=estimator1.fit(
#         inputs={
#             "train": TrainingInput(
#                 s3_data=split_step.properties.ProcessingOutputConfig.Outputs[
#                     "train"
#                 ].S3Output.S3Uri,
#                 content_type="text/csv"
#             ),
#         }
#     ),
#     cache_config=cache_config
# )

/usr/local/lib/python3.8/site-packages/sagemaker/workflow/pipeline_context.py:297: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [39]:


# create_model_step = ModelStep(
#     name="transformer",
#     # display_name="create-model",
#     step_args=skmodel.create(
#         instance_type="ml.m5.large"
#     ),
# )


# transformer = Transformer(
#     model_name=create_model_step.properties.ModelName,
#     base_transform_job_name="transformer",

#     instance_type="ml.c5.xlarge",
#     instance_count=1,
#     assemble_with="Line", 
#     accept="text/csv",
    
#     # output_path=f"s3://mlschool/cohort7/transform",
#     sagemaker_session=pipeline_session
# )

# transform_train_data_step = TransformStep(
#     name="transform-train-data",
#     step_args=transformer.transform(
#         data=split_step.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
#         content_type="text/csv"
#     ),
#     cache_config=cache_config
# )

# transform_validation_data_step = TransformStep(
#     name="transform-validation-data",
#     step_args=transformer.transform(
#         data=split_step.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri,
#         content_type="text/csv"
#     ),
#     cache_config=cache_config
# )

# transform_test_data_step = TransformStep(
#     name="transform-test-data",
#     step_args=transformer.transform(
#         data=split_step.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
#         content_type="text/csv"
#     ),
#     cache_config=cache_config
# )

In [40]:
%%writefile {CODE_FOLDER}/train.py

import os
import argparse

import numpy as np
import pandas as pd
import tensorflow as tf

from pathlib import Path
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD


def train(model_directory, train_path, validation_path, epochs=50, batch_size=32):
    train_files = [file for file in Path(train_path).glob("*.csv")]
    validation_files = [file for file in Path(validation_path).glob("*.csv")]
    
    if len(train_files) == 0 or len(validation_files) == 0:
        raise ValueError("The are no train or validation files")
        
    train_data = [pd.read_csv(file, header=None) for file in train_files]
    X_train = pd.concat(train_data)
    # X_train = pd.read_csv(Path(train_path), header=None)
    y_train = X_train[X_train.columns[-1]]
    X_train.drop(X_train.columns[-1], axis=1, inplace=True)
    
    
    validation_data = [pd.read_csv(file, header=None) for file in validation_files]
    X_validation = pd.concat(validation_data)
    # X_validation = pd.read_csv(Path(validation_path), header=None)
    y_validation = X_validation[X_validation.columns[-1]]
    X_validation.drop(X_validation.columns[-1], axis=1, inplace=True)
    
    model = Sequential([
        Dense(10, input_shape=(X_train.shape[1],), activation="relu"),
        Dense(8, activation="relu"),
        Dense(3, activation="softmax"),
    ])
    
    model.compile(
        optimizer=SGD(learning_rate=0.01),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(
        X_train, 
        y_train, 
        validation_data=(X_validation, y_validation),
        epochs=epochs, 
        batch_size=batch_size,
        verbose=2,
    )

    predictions = np.argmax(model.predict(X_validation), axis=-1)
    print(f"Validation accuracy: {accuracy_score(y_validation, predictions)}")
    
    model_filepath = Path(model_directory) / "001"
    model.save(model_filepath)    
    

if __name__ == "__main__":
    # Any hyperparameters provided by the training job are passed to 
    # the entry point as script arguments. 
    parser = argparse.ArgumentParser()
    parser.add_argument("--epochs", type=int, default=50)
    parser.add_argument("--batch_size", type=int, default=32)
    args, _ = parser.parse_known_args()
    

    train(
        # This is the location where we need to save our model. SageMaker will
        # create a model.tar.gz file with anything inside this directory when
        # the training script finishes.
        model_directory=os.environ["SM_MODEL_DIR"],

        # SageMaker creates one channel for each one of the inputs to the
        # Training Step.
        train_path=os.environ["SM_CHANNEL_TRAIN"],
        validation_path=os.environ["SM_CHANNEL_VALIDATION"],

        epochs=args.epochs,
        batch_size=args.batch_size,
    )

Overwriting code/train.py


In [41]:
from datetime import datetime

from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from sagemaker.experiments import Run

from sagemaker.tuner import HyperparameterTuner
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TuningStep
from sagemaker.parameter import IntegerParameter
from sagemaker.inputs import TrainingInput
from sagemaker.tensorflow import TensorFlow
from sagemaker.workflow.steps import TrainingStep
from sagemaker.sklearn.model import SKLearnModel




tensorflow_estimator = TensorFlow(
    base_job_name="penguins-training",
    entry_point=f"{CODE_FOLDER}/train.py",
    
    hyperparameters={
        "epochs": 50,
        "batch_size": 32,
    },
    
    metric_definitions=[
        {"Name": "loss", "Regex": "loss: ([0-9\\.]+)"},
        {"Name": "accuracy", "Regex": "accuracy: ([0-9\\.]+)"},
        {"Name": "val_loss", "Regex": "val_loss: ([0-9\\.]+)"},
        {"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"},
    ],
    
    framework_version="2.6",
    instance_type="ml.m5.large",
    py_version="py38",
    instance_count=1,
    role=role,
    sagemaker_session=pipeline_session,
)

train_model_step = TrainingStep(
    name="train-model",
    step_args=tensorflow_estimator.fit(
        inputs={
            "train": TrainingInput(
                s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                    "train"
                ].S3Output.S3Uri,
                content_type="text/csv"
            ),
            "validation": TrainingInput(
                s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                    "validation"
                ].S3Output.S3Uri,
                content_type="text/csv"
            )
        }
    ),
    cache_config=cache_config
)

In [86]:
input_data = [[0.958234251, 0.0416116305, 0.000154100155], [0.924666941, 0.0744678229, 0.000865289418], [0.883159637, 0.11454577, 0.00229456788]]
categories = np.array(["Adelie", "Gentoo", "Chinstrap"])
predictions = np.argmax(input_data, axis=-1)
confidence = np.max(input_data, axis=-1)

result = [(categories[pred], conf) for pred, conf in zip(predictions, confidence)]
result

[('Adelie', 0.958234251), ('Adelie', 0.924666941), ('Adelie', 0.883159637)]

In [87]:
%%writefile {CODE_FOLDER}/postprocessor.py

import os
import numpy as np
import pandas as pd
import argparse
import json
import tarfile

from pathlib import Path
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, OrdinalEncoder
from pickle import dump, load

import time
import sys
from io import StringIO

import shutil

import csv
import joblib


from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from pickle import dump


try:
    from sagemaker_containers.beta.framework import (
        content_types,
        encoders,
        env,
        modules,
        transformer,
        worker,
        server,
    )
except ImportError:
    pass

# {
#     "predictions": [[0.821099818, 0.168315798, 0.0105843134], [0.806652844, 0.179276273, 0.0140708638], [0.845787048, 0.148024455, 0.00618851744]]
# }


def input_fn(input_data, content_type):
    print("INPUT_FN", input_data, content_type)
    
    if content_type == "application/json":
        predictions = json.loads(input_data)["predictions"]
        return predictions
    
    else:
        raise ValueError(f"{content_type} is not supported.!")


def output_fn(prediction, accept):
    print("OUTPUT_FN", prediction, accept)
    
    if accept == "application/json":
        result = []
        for p, c in prediction:
            result.append({
                "prediction": p,
                "confidence": c
            })
            
        print("result", result)
        return worker.Response(json.dumps(result), mimetype=accept)
    
    elif accept == 'text/csv':
        
        print("Accept CSV:", prediction)
        
        return worker.Response(encoders.encode(prediction, accept), mimetype=accept)
    else:
        raise RuntimeException(f"{accept} accept type is not supported.")


def predict_fn(input_data, model):
    print("PREDICT_FN", input_data)
    
    categories = model["target_transformer"].named_transformers_["species"].categories_[0]
    print("categories", categories)
    
    predictions = np.argmax(input_data, axis=-1)
    print("predictions", predictions)
    
    
    confidence = np.max(input_data, axis=-1)
    print("confidence", confidence)
    
    result = [(categories[pred], conf) for pred, conf in zip(predictions, confidence)]
    print("result", result)
    
    return result


def model_fn(model_dir):
    """Deserialize fitted model
    """
    
    return {
        "target_transformer": joblib.load(os.path.join(model_dir, "target.joblib")),
        "features_transformer": joblib.load(os.path.join(model_dir, "features.joblib")),
    }

Overwriting code/postprocessor.py


In [88]:
from sagemaker.tensorflow import TensorFlowProcessor
from sagemaker.model import Model
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.predictor import Predictor
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.functions import Join
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.parameters import ParameterFloat
from sagemaker.pipeline import PipelineModel
from sagemaker.sklearn.model import SKLearnModel


skmodel = SKLearnModel(
    model_data=Join(
        on="/", 
        values=[
            preprocess_step.properties.ProcessingOutputConfig.Outputs["model"].S3Output.S3Uri,
            "model.tar.gz"
        ]
    ),
    framework_version="1.2-1",
    sagemaker_session=pipeline_session,
    role=role,
    entry_point="preprocessor.py",
    source_dir=str(CODE_FOLDER),
)   


post_processing_model = SKLearnModel(
    model_data=Join(
        on="/", 
        values=[
            preprocess_step.properties.ProcessingOutputConfig.Outputs["model"].S3Output.S3Uri,
            "model.tar.gz"
        ]
    ),
    framework_version="1.2-1",
    sagemaker_session=pipeline_session,
    role=role,
    entry_point="postprocessor.py",
    source_dir=str(CODE_FOLDER),
)   


model = TensorFlowModel(
    model_data=train_model_step.properties.ModelArtifacts.S3ModelArtifacts,
    framework_version="2.6",
    sagemaker_session=pipeline_session,
    role=role,
)

pipeline_model = PipelineModel(
    name="InferencePipelineModel", 
    role=role, 
    models=[
        skmodel, 
        model,
        post_processing_model
    ],
    sagemaker_session=pipeline_session
)

register_model_step = ModelStep(
    name="PipelineModel",
    step_args=pipeline_model.register(
        model_package_group_name="cohort7",
        approval_status="Approved",
        content_types=["text/csv"],
        response_types=["text/csv"],
        inference_instances=["ml.m5.large"],
        transform_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="TENSORFLOW",
        framework_version="2.6",
    ),
)



INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


In [89]:
cohort7pipeline = Pipeline(
    name="cohort7",
    steps=[
        # split_step,
        preprocess_step,
        # create_model_step,
        # transform_train_data_step,
        # transform_validation_data_step,
        # transform_test_data_step,
        train_model_step,
        register_model_step
    ],
    # pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=pipeline_session
)

cohort7pipeline.upsert(role_arn=role)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:325223348818:pipeline/cohort7',
 'ResponseMetadata': {'RequestId': '556209ca-aa49-4916-a015-b0ede29df5ea',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '556209ca-aa49-4916-a015-b0ede29df5ea',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '75',
   'date': 'Sun, 24 Sep 2023 18:19:11 GMT'},
  'RetryAttempts': 0}}

In [90]:
cohort7pipeline.start()

_PipelineExecution(arn='arn:aws:sagemaker:us-east-1:325223348818:pipeline/cohort7/execution/4f6zfrrfn32v', sagemaker_session=<sagemaker.workflow.pipeline_context.PipelineSession object at 0x7f38b6fc7820>)

In [92]:
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutput, LambdaOutputTypeEnum
from sagemaker.workflow.parameters import ParameterBoolean
from sagemaker.lambda_helper import Lambda
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.s3 import S3Downloader
from sagemaker import ModelPackage
from sagemaker.predictor import Predictor
from sagemaker.workflow.parameters import ParameterInteger

response = sagemaker_client.list_model_packages(
    ModelPackageGroupName="cohort7",
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=1,
)

package = response["ModelPackageSummaryList"][0] if response["ModelPackageSummaryList"] else None
package

{'ModelPackageGroupName': 'cohort7',
 'ModelPackageVersion': 8,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:325223348818:model-package/cohort7/8',
 'CreationTime': datetime.datetime(2023, 9, 24, 18, 19, 17, 680000, tzinfo=tzlocal()),
 'ModelPackageStatus': 'Completed',
 'ModelApprovalStatus': 'Approved'}

In [93]:
model_package = ModelPackage(
    model_package_arn=package["ModelPackageArn"], 
    sagemaker_session=sagemaker_session,
    role=role, 
)

model_package.deploy(
    endpoint_name="cohort7", 
    initial_instance_count=1, 
    instance_type="ml.m5.large"
)

INFO:sagemaker:Creating model with name: cohort7-2023-09-24-18-19-23-762
INFO:sagemaker:Creating endpoint-config with name cohort7
INFO:sagemaker:Creating endpoint with name cohort7


------!

In [95]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

# payload = "M, 0.44, 0.365, 0.125, 0.516, 0.2155, 0.114, 0.155"

payload = "Torgersen, 39.5, 17.4, 186, 3800, FEMALE" # Adelie

# Adelie,Torgersen,39.5,17.4,186,3800,FEMALE
# Adelie,Torgersen,40.3,18,195,3250,FEMALE
# Adelie,Torgersen,NA,NA,NA,NA,NA
# Adelie,Torgersen,36.7,19.3,193,3450,FEMALE
# Adelie,Torgersen,39.3,20.6,190,3650,MALE


predictor = Predictor(
    endpoint_name="cohort7", 
    sagemaker_session=sagemaker_session, 
    serializer=CSVSerializer()
)

# print(predictor.predict(payload, initial_args={"ContentType": "text/csv"}))


data = pd.read_csv("data.csv")
data = data.drop("species", axis=1)

pred_count = 10
payload = data.iloc[:pred_count].to_csv(header=False, index=False)
p = predictor.predict(payload, initial_args={"ContentType": "text/csv"})
print(p.decode("utf-8"))

[{"prediction": "Adelie", "confidence": 0.958234251}, {"prediction": "Adelie", "confidence": 0.924666941}, {"prediction": "Adelie", "confidence": 0.883159637}, {"prediction": "Adelie", "confidence": 0.471390933}, {"prediction": "Adelie", "confidence": 0.966129959}, {"prediction": "Adelie", "confidence": 0.959097147}, {"prediction": "Adelie", "confidence": 0.952538788}, {"prediction": "Adelie", "confidence": 0.940574}, {"prediction": "Adelie", "confidence": 0.970270753}, {"prediction": "Adelie", "confidence": 0.897611558}]


In [96]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: cohort7
INFO:sagemaker:Deleting endpoint with name: cohort7


## Step 2 - Testing the Preprocessing Script

We can now load the script we just created and run it locally to ensure it outputs every file we need. In this case, we can call the `preprocess()` function with the local directory and the local copy of the dataset.

# Session 3 - Building a Model

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) we built in the previous session with a step to train a model. We'll explore the [Training](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training) and the [Tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-tuning) steps. 



We'll introduce [Amazon SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) and use them during training. For more information about this topic, check the [SageMaker Experiments' SDK documentation](https://sagemaker.readthedocs.io/en/v2.174.0/experiments/sagemaker.experiments.html).

In [428]:
from datetime import datetime

from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from sagemaker.experiments import Run

from sagemaker.tuner import HyperparameterTuner
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TuningStep
from sagemaker.parameter import IntegerParameter
from sagemaker.inputs import TrainingInput
from sagemaker.tensorflow import TensorFlow
from sagemaker.workflow.steps import TrainingStep

## Step 1 - Creating the Training Script

This script is responsible for training the neural network on the train data, validating the model, and saving it so we can later use it.

In [429]:
%%writefile {CODE_FOLDER}/train.py

import os
import argparse

import numpy as np
import pandas as pd
import tensorflow as tf

from pathlib import Path
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD


def train(model_directory, train_path, validation_path, epochs=50, batch_size=32):
    X_train = pd.read_csv(Path(train_path) / "train.csv")
    y_train = X_train[X_train.columns[-1]]
    X_train.drop(X_train.columns[-1], axis=1, inplace=True)
    
    X_validation = pd.read_csv(Path(validation_path) / "validation.csv")
    y_validation = X_validation[X_validation.columns[-1]]
    X_validation.drop(X_validation.columns[-1], axis=1, inplace=True)
    
    model = Sequential([
        Dense(10, input_shape=(X_train.shape[1],), activation="relu"),
        Dense(8, activation="relu"),
        Dense(3, activation="softmax"),
    ])
    
    model.compile(
        optimizer=SGD(learning_rate=0.01),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(
        X_train, 
        y_train, 
        validation_data=(X_validation, y_validation),
        epochs=epochs, 
        batch_size=batch_size,
        verbose=2,
    )

    predictions = np.argmax(model.predict(X_validation), axis=-1)
    print(f"Validation accuracy: {accuracy_score(y_validation, predictions)}")
    
    model_filepath = Path(model_directory) / "001"
    model.save(model_filepath)    
    

if __name__ == "__main__":
    # Any hyperparameters provided by the training job are passed to 
    # the entry point as script arguments. 
    parser = argparse.ArgumentParser()
    parser.add_argument("--epochs", type=int, default=50)
    parser.add_argument("--batch_size", type=int, default=32)
    args, _ = parser.parse_known_args()
    

    train(
        # This is the location where we need to save our model. SageMaker will
        # create a model.tar.gz file with anything inside this directory when
        # the training script finishes.
        model_directory=os.environ["SM_MODEL_DIR"],

        # SageMaker creates one channel for each one of the inputs to the
        # Training Step.
        train_path=os.environ["SM_CHANNEL_TRAIN"],
        validation_path=os.environ["SM_CHANNEL_VALIDATION"],

        epochs=args.epochs,
        batch_size=args.batch_size,
    )

Overwriting code/train.py


## Step 2 - Testing the Training Script

Let's test the script we just created by running it locally.

In [430]:
from preprocessor import preprocess
from train import train


with tempfile.TemporaryDirectory() as directory:
    # First, we preprocess the data and create the 
    # dataset splits.
    preprocess(
        base_directory=directory, 
        data_filepath=DATA_FILEPATH
    )

    # Then, we train a model using the train and 
    # validation splits.
    train(
        model_directory=directory, 
        train_path=Path(directory) / "train", 
        validation_path=Path(directory) / "validation",
        epochs=10
    )

Epoch 1/10
8/8 - 1s - loss: 1.0120 - accuracy: 0.7531 - val_loss: 1.0090 - val_accuracy: 0.7451
Epoch 2/10
8/8 - 0s - loss: 1.0006 - accuracy: 0.7824 - val_loss: 1.0001 - val_accuracy: 0.7255
Epoch 3/10
8/8 - 0s - loss: 0.9893 - accuracy: 0.7908 - val_loss: 0.9905 - val_accuracy: 0.7451
Epoch 4/10
8/8 - 0s - loss: 0.9772 - accuracy: 0.7950 - val_loss: 0.9812 - val_accuracy: 0.7647
Epoch 5/10
8/8 - 0s - loss: 0.9653 - accuracy: 0.8075 - val_loss: 0.9711 - val_accuracy: 0.7647
Epoch 6/10
8/8 - 0s - loss: 0.9522 - accuracy: 0.8075 - val_loss: 0.9610 - val_accuracy: 0.7843
Epoch 7/10
8/8 - 0s - loss: 0.9384 - accuracy: 0.8033 - val_loss: 0.9501 - val_accuracy: 0.7843
Epoch 8/10
8/8 - 0s - loss: 0.9238 - accuracy: 0.8033 - val_loss: 0.9385 - val_accuracy: 0.7843
Epoch 9/10
8/8 - 0s - loss: 0.9089 - accuracy: 0.8033 - val_loss: 0.9267 - val_accuracy: 0.7843
Epoch 10/10
8/8 - 0s - loss: 0.8935 - accuracy: 0.8033 - val_loss: 0.9145 - val_accuracy: 0.7647
Validation accuracy: 0.7647058823529411

INFO:tensorflow:Assets written to: /tmp/tmp3mbewkwx/001/assets


## Step 3 - Setting up a Training Step

We can now create a [Training Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training) that we can add to the pipeline. This Training Step will create a SageMaker Training Job in the background, run the training script, and upload the output to S3. Check the [TrainingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TrainingStep) SageMaker's SDK documentation for more information. 

SageMaker uses the concept of an [Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) to handle end-to-end training and deployment tasks. For this example, we will use the built-in [TensorFlow Estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-estimator) to run the training script we wrote before. The [Docker Registry Paths and Example Code](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html) page contains information about the available framework versions for each region. Here, you can also check the available SageMaker [Deep Learning Container images](https://github.com/aws/deep-learning-containers/blob/master/available_images.md).

Notice the list of hyperparameters defined below. SageMaker will pass these hyperparameters as arguments to the entry point of the training script.

We are going to use [SageMaker Experiments](https://sagemaker.readthedocs.io/en/v2.174.0/experiments/sagemaker.experiments.html) to log information from the Training Job. For more information, check [Manage Machine Learning with Amazon SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html). The list of metric definitions will tell SageMaker which metrics to track and how to parse them from the Training Job logs.

In [431]:
estimator = TensorFlow(
    base_job_name="penguins-training",
    entry_point=f"{CODE_FOLDER}/train.py",
    
    hyperparameters={
        "epochs": 50,
        "batch_size": 32,
    },
    
    metric_definitions=[
        {"Name": "loss", "Regex": "loss: ([0-9\\.]+)"},
        {"Name": "accuracy", "Regex": "accuracy: ([0-9\\.]+)"},
        {"Name": "val_loss", "Regex": "val_loss: ([0-9\\.]+)"},
        {"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"},
    ],
    
    framework_version="2.6",
    instance_type="ml.m5.large",
    py_version="py38",
    instance_count=1,
    role=role,
    sagemaker_session=pipeline_session,
)

We can now create the [TrainingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TrainingStep) using the estimator we defined before.

This step will receive the train and validation split from the preprocessing step as inputs. Notice how we reference both splits using preprocess step. This creates a dependency between the Training and Processing Steps.

Here, we are using two input channels, `train` and `validation`. SageMaker will automatically create an environment variable corresponding to each of these channels following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAIN`: This environment variable will contain the path to the data in the `train` channel
* `SM_CHANNEL_VALIDATION`: This environment variable will contain the path to the data in the `validation` channel

In [432]:
train_model_step = TrainingStep(
    name="train-model",
    step_args=estimator.fit(
        inputs={
            "train": TrainingInput(
                s3_data=preprocess_data_step.properties.ProcessingOutputConfig.Outputs[
                    "train"
                ].S3Output.S3Uri,
                content_type="text/csv"
            ),
            "validation": TrainingInput(
                s3_data=preprocess_data_step.properties.ProcessingOutputConfig.Outputs[
                    "validation"
                ].S3Output.S3Uri,
                content_type="text/csv"
            )
        }
    ),
    cache_config=cache_config
)

/usr/local/lib/python3.8/site-packages/sagemaker/workflow/pipeline_context.py:297: UserWarning:

Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.



## Step 4 - Setting up a Tuning Step

Let's now create a [Tuning Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-tuning). This Tuning Step will create a SageMaker Hyperparameter Tuning Job in the background and use the training script to train different model variants and choose the best one. Check the [TuningStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep) SageMaker's SDK documentation for more information.

The Tuning Step requires a [HyperparameterTuner](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html) reference to configure the Hyperparameter Tuning Job.

Here is the configuration that we'll use to find the best model:

1. `objective_metric_name`: This is the name of the metric the tuner will use to determine the best model.
2. `objective_type`: This is the objective of the tuner. Should it "Minimize" the metric or "Maximize" it? In this example, since we are using the validation accuracy of the model, we want the objective to be "Maximize." If we were using the loss of the model, we would set the objective to "Minimize."
3. `metric_definitions`: Defines how the tuner will determine the metric's value by looking at the output logs of the training process.

The tuner expects the list of the hyperparameters you want to explore. You can use subclasses of the [Parameter](https://sagemaker.readthedocs.io/en/stable/api/training/parameter.html#sagemaker.parameter.ParameterRange) class to specify different types of hyperparameters. This example explores different values for the `epochs` hyperparameter.

Finally, you can control the number of jobs and how many of them will run in parallel using the following two arguments:

* `max_jobs`: Defines the maximum total number of training jobs to start for the hyperparameter tuning job.
* `max_parallel_jobs`: Defines the maximum number of parallel training jobs to start.

In [433]:
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name = "val_accuracy",
    objective_type="Maximize",
    hyperparameter_ranges = {
        "epochs": IntegerParameter(10, 50),
    },
    metric_definitions = [
        {"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"}
    ],
    max_jobs=3,
    max_parallel_jobs=3,
)

We can now create the [TuningStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep). 

This step will use the tuner we configured before and will receive the train and validation split from the preprocessing step as inputs.

In [434]:
tune_model_step = TuningStep(
    name = "tune-model",
    step_args=tuner.fit(
        inputs={
            "train": TrainingInput(
                s3_data=preprocess_data_step.properties.ProcessingOutputConfig.Outputs[
                    "train"
                ].S3Output.S3Uri,
                content_type="text/csv"
            ),
            "validation": TrainingInput(
                s3_data=preprocess_data_step.properties.ProcessingOutputConfig.Outputs[
                    "validation"
                ].S3Output.S3Uri,
                content_type="text/csv"
            )
        },
    ),
    cache_config=cache_config
)

## Step 5 - Switching Between Training and Tuning

We could use a [Training Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training) or use a [Tuning Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-tuning) to create the model.

In this notebook, we will alternate between both methods and use the `USE_TUNING_STEP` flag to indicate which approach we want to run.

In [435]:
USE_TUNING_STEP = False

## Step 6 - Setting up the Pipeline

We can now define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.

In [436]:
session3_pipeline = Pipeline(
    name="penguins-session3",
    parameters=[
        dataset_location
    ],
    steps=[
        preprocess_data_step, 
        tune_model_step if USE_TUNING_STEP else train_model_step
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=pipeline_session
)

session3_pipeline.upsert(role_arn=role)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:325223348818:pipeline/penguins-session3',
 'ResponseMetadata': {'RequestId': 'ac20e8b9-ff2e-4e8c-b3bc-e8e7c5a9cafa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ac20e8b9-ff2e-4e8c-b3bc-e8e7c5a9cafa',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '85',
   'date': 'Sat, 23 Sep 2023 19:07:54 GMT'},
  'RetryAttempts': 0}}

## Questions

Answering these questions will help you understand the material we discussed during this session. Notice that each question could have one or more correct answers.

<div style="margin: 30px 0 10px 0;"><span style="font-size: 1.1em; padding:4px; background-color: #b8bf9f; color: #000;"><strong>Question 3.1</strong></span></div>

Why do we use the `SparseCategoricalCrossentropy` loss function to train our model instead of the `CategoricalCrossentropy` function?

1. Because our target column are integer values.
2. Because our target column is one-hot encoded.
3. Because our target column are categorical values.
4. Because there are a lot of sparse values in our dataset.


<div style="margin: 30px 0 10px 0;"><span style="font-size: 1.1em; padding:4px; background-color: #b8bf9f; color: #000;"><strong>Question 3.2</strong></span></div>

When a Training Job finishes, SageMaker automatically uploads the model to S3. Which of the following statements about this process is correct?

1. SageMaker automatically creates a `model.tar.gz` file with the entire content of the `/opt/ml/model` directory.
2. SageMaker automatically creates a `model.tar.gz` file with any files inside the `/opt/ml/model` directory as long as those files belong to the model we trained.
3. SageMaker automatically creates a `model.tar.gz` file with any new files created inside the container by the training script.
4. SageMaker automatically creates a `model.tar.gz` file with the content of the output folder configured in the training script.


<div style="margin: 30px 0 10px 0;"><span style="font-size: 1.1em; padding:4px; background-color: #b8bf9f; color: #000;"><strong>Question 3.3</strong></span></div>

Our pipeline uses "file mode" to provide the Training Job access to the dataset. When using file mode, SageMaker downloads the training data from S3 to a local directory in the training container. Imagine we have a large dataset and don't want to wait for SageMaker to download every time we want to train a model. How can we solve this problem?

1. We can train our model with a smaller portion of the dataset.
2. We can increase the number of instances and train many models in parallel.
3. We can use "fast file mode" to get file system access to S3.
4. We can use "pipe mode" to stream data directly from S3 into the training container.


<div style="margin: 30px 0 10px 0;"><span style="font-size: 1.1em; padding:4px; background-color: #b8bf9f; color: #000;"><strong>Question 3.4</strong></span></div>

Which of the following statements are true about the usage of `max_jobs` and `max_parallel_jobs` when running a Hyperparameter Tuning Job?

1. `max_jobs` represents the maximum total number of Training Jobs that the Hyperparameter Tuning Job will start. 
2. `max_parallel_jobs` represents the maximum total number of Training Jobs that will run in parallel at any given time during a Hyperparameter Tuning Job.
3. `max_parallel_jobs` can never be larger than `max_jobs`.
4. `max_jobs` can never be larger than `max_parallel_jobs`.


<div style="margin: 30px 0 10px 0;"><span style="font-size: 1.1em; padding:4px; background-color: #b8bf9f; color: #000;"><strong>Question 3.5</strong></span></div>

Which of the following statements are true about tuning hyperparameters as part of a pipeline?

1. Hyperparameter Tuning Jobs that don't use Amazon algorithms require a regular expression to extract the objective metric from the logs.
2. When using a Tuning Step as part of a pipeline, SageMaker will create as many Hyperparameter Tuning Jobs as specified by the `HyperparameterTuner.max_jobs` attribute.
3. Hyperparameter Tuning Jobs support Bayesian, Grid Search, and Random Search strategies.
4. Using a Tuning Step is more expensive than using a Training Step.


## Assignments


* <span style="padding:4px; background-color: #f2a68a; color: #000;"><strong>Assignment 3.1</strong></span> The training script is using a hard-coded learning rate value to train the model. Modify the code to accept the learning rate as a parameter that we can control from outside the script.

* <span style="padding:4px; background-color: #f2a68a; color: #000;"><strong>Assignment 3.2</strong></span> We currently define the number of epochs to train the model as a constant that we pass to the Estimator using the list of hyperparameters. Replace this constant with a new Pipeline Parameter named `training_epochs`. You'll need to specify this new parameter when creating the Pipeline.

* <span style="padding:4px; background-color: #f2a68a; color: #000;"><strong>Assignment 3.3</strong></span> The goal of the current tuning process is to find the model with the highest validation accuracy. Modify the code to focus on the model with the lowest training loss.

* <span style="padding:4px; background-color: #f2a68a; color: #000;"><strong>Assignment 3.4</strong></span> Compare the validation accuracy of the last few runs of your experiments. To analyze experiment runs, select the experiment in the SageMaker Studio Experiments UI, select the runs that you want to compare, and click on the Analyze button. Check [Create an Amazon SageMaker Experiment](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments-create.html) documentation for more information.

* <span style="padding:4px; background-color: #f2a68a; color: #000;"><strong>Assignment 3.5</strong></span> Modify the pipeline you created for the "Pipeline of Digits" project and add a Training Step. This Training Step should receive the train and validation splits from the Preprocessing step.


# Session 4  - Evaluating and Registering The Model

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with a step to evaluate and register the model if it reaches a predefined accuracy threshold. We'll use a [Processing Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing) with a [ScriptProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor) running TensorFlow to execute an evaluation script. We'll use a [Condition Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition) to determine whether the model's accuracy is above a threshold and a [Model Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-model) to register the model. To learn more about the Model Registry, check [Register and Deploy Models with Model Registry](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry.html).

In [437]:
import tarfile

from sagemaker.tensorflow import TensorFlowProcessor
from sagemaker.model import Model
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.predictor import Predictor
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.functions import Join
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.parameters import ParameterFloat


MODEL_PACKAGE_GROUP = "penguins"

## Step 1 - Evaluating the Model

This script is responsible for loading the model we created and evaluating it on the test set. Before finishing, this script will generate an evaluation report of the model.

In [438]:
%%writefile {CODE_FOLDER}/evaluation.py

import os
import json
import tarfile
import numpy as np
import pandas as pd

from pathlib import Path
from tensorflow import keras
from sklearn.metrics import accuracy_score


MODEL_PATH = "/opt/ml/processing/model/"
TEST_PATH = "/opt/ml/processing/test/"
OUTPUT_PATH = "/opt/ml/processing/evaluation/"


def evaluate(model_path, test_path, output_path):
    # The first step is to extract the model package so we can load 
    # it in memory.
    with tarfile.open(Path(model_path) / "model.tar.gz") as tar:
        tar.extractall(path=Path(model_path))
        
    model = keras.models.load_model(Path(model_path) / "001")
    
    X_test = pd.read_csv(Path(test_path) / "test.csv")
    y_test = X_test[X_test.columns[-1]]
    X_test.drop(X_test.columns[-1], axis=1, inplace=True)
    
    predictions = np.argmax(model.predict(X_test), axis=-1)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Test accuracy: {accuracy}")

    # Let's create an evaluation report using the model accuracy.
    evaluation_report = {
        "metrics": {
            "accuracy": {
                "value": accuracy
            },
        },
    }
    
    Path(output_path).mkdir(parents=True, exist_ok=True)
    with open(Path(output_path) / "evaluation.json", "w") as f:
        f.write(json.dumps(evaluation_report))
        
        
if __name__ == "__main__":
    evaluate(
        model_path=MODEL_PATH, 
        test_path=TEST_PATH,
        output_path=OUTPUT_PATH
    )

Overwriting code/evaluation.py


## Step 2 - Testing the Evaluation Script

Let's test the script we just created by running it locally.

In [ ]:
from preprocessor import preprocess
from train import train
from evaluation import evaluate


with tempfile.TemporaryDirectory() as directory:
    preprocess(
        base_directory=directory, 
        data_filepath=DATA_FILEPATH
    )

    train(
        model_directory=directory, 
        train_path=Path(directory) / "train", 
        validation_path=Path(directory) / "validation",
        epochs=10
    )
    
    # After training a model, we need to prepare a package just like
    # SageMaker would. This package is what the evaluation script is
    # expecting as an input.
    with tarfile.open(Path(directory) / "model.tar.gz", "w:gz") as tar:
        tar.add(Path(directory) / "001", arcname="001")
        
    
    # We can now call the evaluation script.
    evaluate(
        model_path=directory, 
        test_path=Path(directory) / "test",
        output_path=Path(directory) / "evaluation",
    )

## Step 3 - Setting up a Processing Step

To run the evaluation script, we can use a [Processing Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing). In this case, we'll use the [TensorFlowProcessor](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job-frameworks-tensorflow.html) because we need access to TensorFlow. 

The inputs of this Processing Step will be the model we created and the test set. The output will be the evaluation report file.

We can use the `USE_TUNING_STEP` flag to determine whether we created the model using a Training Step or a Tuning Step. In case we are using the Tuning Step, we can use the [TuningStep.get_top_model_s3_uri()](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep.get_top_model_s3_uri) function to get the model artifacts from the top performing training job of the Hyperparameter Tuning Job.

The [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) lets us specify a list of [PropertyFile](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.properties.PropertyFile) instances from the output of the job. We can use this to map the evaluation report generated in the evaluation script. Check [How to Build and Manage Property Files](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-propertyfile.html) for more information.

In [ ]:
tensorflow_processor = TensorFlowProcessor(
    base_job_name="penguins-evaluation-processor",
    framework_version="2.6",
    py_version="py38",
    instance_type="ml.m5.large",
    instance_count=1,
    role=role,
    sagemaker_session=pipeline_session
)

# We want to map the evaluation report that we generate inside
# the evaluation script so we can later reference it.
evaluation_report = PropertyFile(
    name="evaluation-report",
    output_name="evaluation",
    path="evaluation.json"
)


# Notice how this step uses the model generated by the tuning or training
# step, and the test set generated by the preprocessing step.
evaluate_model_step = ProcessingStep(
    name="evaluate-model",
    step_args=tensorflow_processor.run(
        inputs=[
            ProcessingInput(
                source=preprocess_data_step.properties.ProcessingOutputConfig.Outputs[
                    "test"
                ].S3Output.S3Uri,
                destination="/opt/ml/processing/test"
            ),
            ProcessingInput(
                source=(
                    tune_model_step.get_top_model_s3_uri(top_k=0, s3_bucket=pipeline_session.default_bucket()) 
                    if USE_TUNING_STEP 
                    else train_model_step.properties.ModelArtifacts.S3ModelArtifacts
                ),
                destination="/opt/ml/processing/model",
            )
        ],
        outputs=[
            ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
        ],
        code=f"{CODE_FOLDER}/evaluation.py"
    ),
    property_files=[evaluation_report],
    cache_config=cache_config
)

## Step 4 - Configuring the Model Metrics

When we register a model, we can specify a set of [ModelMetrics](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_metrics.ModelMetrics). We can use the evaluation report we generated during the Evaluation step to populate these statistics.

In [ ]:
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(on="/", values=[
            evaluate_model_step.arguments['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri'],
            "evaluation.json"]
        ),
        content_type="application/json",
    )
)

## Step 5 - Registering the Model

We can now create a [Model Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-model) to register the model. Check the [ModelStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.model_step.ModelStep) SageMaker's SDK documentation for more information. We want to create a new version of the model and register it in the Model Registry. Check [Register a Model Version](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-version.html) for more information about model registration.

The model we trained uses TensorFlow, so we can use the built-in [TensorFlowModel](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-serving-model) class to create an instance of the model.

In [ ]:
model = TensorFlowModel(
    model_data=(
        tune_model_step.get_top_model_s3_uri(top_k=0, s3_bucket=pipeline_session.default_bucket())
        if USE_TUNING_STEP
        else train_model_step.properties.ModelArtifacts.S3ModelArtifacts
    ),
    framework_version="2.6",
    sagemaker_session=pipeline_session,
    role=role,
)

register_model_step = ModelStep(
    name="register-model",
    step_args=model.register(
        model_package_group_name=MODEL_PACKAGE_GROUP,
        model_metrics=model_metrics,
        approval_status="Approved",
        content_types=["text/csv"],
        response_types=["text/csv"],
        inference_instances=["ml.m5.large"],
        transform_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="TENSORFLOW",
        framework_version="2.6",
    ),
)

## Step 6 - Setting up a Condition Step

We only want to register a new model if its accuracy exceeds a predefined threshold. We can use a [Condition Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition) together with the evaluation report we generated to accomplish this. Check the [ConditionStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#conditionstep) SageMaker's SDK documentation for more information. In this example, we will use a [ConditionGreaterThanOrEqualTo](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.conditions.ConditionGreaterThanOrEqualTo) condition to compare the model's accuracy with the threshold. Look at the [Conditions](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_building_pipeline.html#conditions) section in the documentation for more information about the types of supported conditions.

If the model's accuracy is not greater than or equal our threshold, we will send the pipeline to a [Fail Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-fail) with the appropriate error message. Check the [FailStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.fail_step.FailStep) SageMaker's SDK documentation for more information.

We are going to use a new [Pipeline Parameter](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-parameters.html) in our pipeline to specify the minimum accuracy that the model should reach for it to be registered.

In [ ]:
accuracy_threshold = ParameterFloat(
    name="accuracy_threshold", 
    default_value=0.70
)

fail_step = FailStep(
    name="fail",
    error_message=Join(
        on=" ", 
        values=[
            "Execution failed because the model's accuracy was lower than", 
            accuracy_threshold
        ]
    ),
)

condition_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=evaluate_model_step.name,
        property_file=evaluation_report,
        json_path="metrics.accuracy.value"
    ),
    right=accuracy_threshold
)

condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[condition_gte],
    if_steps=[register_model_step],
    else_steps=[fail_step], 
)

## Step 7 - Setting up the Pipeline

We can now define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.

In [ ]:
session4_pipeline = Pipeline(
    name="penguins-session4",
    parameters=[
        dataset_location,
        accuracy_threshold,
    ],
    steps=[
        preprocess_data_step, 
        tune_model_step if USE_TUNING_STEP else train_model_step, 
        evaluate_model_step,
        condition_step
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=pipeline_session
)

session4_pipeline.upsert(role_arn=role)

## Questions

Answering these questions will help you understand the material we discussed during this session. Notice that each question could have one or more correct answers.


<div style="margin: 30px 0 10px 0;"><span style="font-size: 1.1em; padding:4px; background-color: #b8bf9f; color: #000;"><strong>Question 4.1</strong></span></div>

When registering a model in the Model Registry, we can specify a set of metrics that will be stored with the model. Which of the following are some of the metrics supported by SageMaker?

1. Metrics that measure the bias in a model.
2. Metrics that help explain a model.
3. Metrics that measure the quality of the input data for a model.
4. Metrics that measure the quality of a model.


<div style="margin: 30px 0 10px 0;"><span style="font-size: 1.1em; padding:4px; background-color: #b8bf9f; color: #000;"><strong>Question 4.2</strong></span></div>

We use the `Join` function to build the error message for the Fail Step. Imagine we want to build a Amazon S3 URI. What would be the output of executing `Join(on='/', values=['s3:/', "mlschool", "/", "12345"])`?

1. The output will be `s3://mlschool/12345`
2. The output will be `s3://mlschool/12345/`
3. The output will be `s3://mlschool//12345`
4. The output will be `s3:/mlschool//12345`


<div style="margin: 30px 0 10px 0;"><span style="font-size: 1.1em; padding:4px; background-color: #b8bf9f; color: #000;"><strong>Question 4.3</strong></span></div>

Which of the following statements are correct about the Condition Step in SageMaker:

1. `ConditionComparison` is a supported condition type.
2. `ConditionIn` is a supported condition type.
3. When using multiple conditions together, the step will succeed if at least one of the conditions returns True.
4. When using multiple conditions together, every one of them has to return True for the step to succeed.


<div style="margin: 30px 0 10px 0;"><span style="font-size: 1.1em; padding:4px; background-color: #b8bf9f; color: #000;"><strong>Question 4.4</strong></span></div>

Imagine we use a Tuning Step to run 100 Training Jobs. The best model should have the higest validation accuracy, but we made a mistake and used "Minimize" as the objective type instead of "Maximize." The consequence is that the index of our best model is 100 instead of 0. How can we retrieve the best model from the Tuning Step?

1. We can use `TuningStep.get_top_model_s3_uri(top_k=0)` to retrieve the best model.
2. We can use `TuningStep.get_top_model_s3_uri(top_k=100)` to retrieve the best model.
3. We can use `TuningStep.get_bottom_model_s3_uri(top_k=0)` to retrieve the best model.
4. In this example, we can't retrieve the best model.

<div style="margin: 30px 0 10px 0;"><span style="font-size: 1.1em; padding:4px; background-color: #b8bf9f; color: #000;"><strong>Question 4.5</strong></span></div>


If the accuracy of the model is above the threshold, our pipeline registers it in the Model Registry. Which of the following functions are related to the Model Registry?

1. Model versioning: We can use the Model Registry to track different versions of a model, especially as it gets updated or refined over time.
2. Model deployment: We can initiate the deployment of a model right from the Model Registry.
3. Model metrics: The Model Registry provides insights about a particular model through the registration of metrics.
4. Model features: The Model Registry lists every feature that was used to build the model.

## Assignments

* <span style="padding:4px; background-color: #f2a68a; color: #000;"><strong>Assignment 4.1</strong></span> Instead of running the entire pipeline from start to finish, sometimes you may only need to iterate over particular steps. SageMaker Pipelines supports [Selective Execution for Pipeline Steps](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-selective-ex.html). In this assignment you will use Selective Execution to only run the Training Step using a different number of epochs. To configure the new value, use the Pipeline Property `training_epochs` that you added during the assignments from the previous session. [Unlocking efficiency: Harnessing the power of Selective Execution in Amazon SageMaker Pipelines](https://aws.amazon.com/blogs/machine-learning/unlocking-efficiency-harnessing-the-power-of-selective-execution-in-amazon-sagemaker-pipelines/) is a great article that explains this feature.

* <span style="padding:4px; background-color: #f2a68a; color: #000;"><strong>Assignment 4.2</strong></span> The current pipeline uses a Training Step or a Tuning Step to build a model. Modify the pipeline to use both steps at the same time and register only the best model coming out from them.

* <span style="padding:4px; background-color: #f2a68a; color: #000;"><strong>Assignment 4.3</strong></span> The evaluation script loads the model from a hard-coded "001" directory. Instead of doing this, let's supply this directory name as an argument to the Processing Job. To do this, check the `arguments` parameter of the [Processor.run](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.Processor.run) function.

* <span style="padding:4px; background-color: #f2a68a; color: #000;"><strong>Assignment 4.4</strong></span> The evaluation script produces an evaluation report containing the accuracy of the model. Extend the evaluation report by adding the precision of the model. We want this new metric to show under the Model Quality section in the Model Registry.

* <span style="padding:4px; background-color: #f2a68a; color: #000;"><strong>Assignment 4.5</strong></span> Modify the SageMaker Pipeline you created for the "Pipeline of Digits" project and add the necessary steps to evaluate and register the model.

# Session 5 - Deploying The Model

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with a step to deploy the model to an endpoint. We'll use a [Lambda Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-lambda) to create an endpoint and deploy the model. To control the endpoint's inputs and outputs, we'll modify the model's assets to include code that customizes the processing of a request. 

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutput, LambdaOutputTypeEnum
from sagemaker.workflow.parameters import ParameterBoolean
from sagemaker.lambda_helper import Lambda
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.s3 import S3Downloader
from sagemaker import ModelPackage
from sagemaker.predictor import Predictor
from sagemaker.workflow.parameters import ParameterInteger

ENDPOINT = "penguins-endpoint"

## Step 1 - Deploy Latest Model From Registry

Let's deploy the latest model from the Model Registry using SageMaker's SDK.

To deploy a model from the Model Registry, we need to find its model package ARN. Let's query the list of approved models and get the latest one.

In [ ]:
response = sagemaker_client.list_model_packages(
    ModelPackageGroupName=MODEL_PACKAGE_GROUP,
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=1,
)

package = response["ModelPackageSummaryList"][0] if response["ModelPackageSummaryList"] else None
package

Using the ARN of the model package from the Model Registry, we can deploy the model by creating a [ModelPackage](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.ModelPackage) instance and calling its `deploy()` function. The model information lives in the Model Registry, so we don't need to specify anything else.

<div class="alert" style="background-color:#0066cc;">Uncomment the <code style="background-color:#0066cc;">%%script</code> cell magic line to execute this cell.</div>

In [ ]:
%%script false --no-raise-error

model_package = ModelPackage(
    model_package_arn=package["ModelPackageArn"], 
    sagemaker_session=sagemaker_session,
    role=role, 
)

model_package.deploy(
    endpoint_name=ENDPOINT, 
    initial_instance_count=1, 
    instance_type="ml.m5.large"
)

Let's test the endpoint to make sure it works. Notice the payload we need to provide the model is in CSV format. The model expects data that's already transformed. We can't provide the original data from our dataset because the model will not work with it.

<div class="alert" style="background-color:#0066cc;">Uncomment the <code style="background-color:#0066cc;">%%script</code> cell magic line to execute this cell.</div>

In [ ]:
%%script false --no-raise-error

# We'll send this payload to the endpoint. Notice how each line contains
# the information of a penguin. The endpoint will return the predictions
# for each of these lines.
payload = """
0.6569590202313976, -1.0813829646495108, 1.2097102831892812, 0.9226343641317372, 1.0, 0.0, 0.0
-0.7751048801481084, 0.8822689351285553,  -1.2168066120762704, 0.9226343641317372, 0.0, 1.0, 0.0
-0.837387834894918, 0.3386660813829646, -0.26237731892812, -1.92351941317372, 0.0, 0.0, 1.0
"""

# We can now send the request to the endpoint and process the response.
predictor = Predictor(endpoint_name=ENDPOINT)
response = predictor.predict(payload, initial_args={"ContentType": "text/csv"})
response = json.loads(response.decode("utf-8"))

print(json.dumps(response, indent=2))
print(f"\nSpecies: {np.argmax(response['predictions'], axis=1)}")

We can now delete the endpoint using the predictor.

<div class="alert" style="background-color:#0066cc;">Uncomment the <code style="background-color:#0066cc;">%%script</code> cell magic line to execute this cell.</div>

In [ ]:
%%script false --no-raise-error

predictor.delete_endpoint()

## Step 2 - Preparing the Inference Code

Deploying the model we trained directly to an endpoint doesn't lets us control the data that goes in and comes out of the endpoint. Fortunately, SageMaker allows us to include an `inference.py` file with the model assets from where we can control how the endpoint works. You can see more information about how this works by checking the [SageMaker TensorFlow Serving Container](https://github.com/aws/sagemaker-tensorflow-serving-container) documentation.

We want our endpoint to handle unprocessed data in JSON format and return the penguin's species. Here is an example of the payload we want the endpoint to support:

```
{
    "island": "Biscoe",
    "culmen_length_mm": 48.6,
    "culmen_depth_mm": 16.0,
    "flipper_length_mm": 230.0,
    "body_mass_g": 5800.0,
}
```

And here is an example of the output we'd like to get from the endpoint:

```
{
    "prediction": "Adelie", 
    "confidence": 0.802672
}
```

Let's start by setting up a local folder where we will create the `inference.py` script.

In [ ]:
ENDPOINT_CODE_FOLDER = CODE_FOLDER / "endpoint"
Path(ENDPOINT_CODE_FOLDER).mkdir(parents=True, exist_ok=True)
sys.path.append(f"./{ENDPOINT_CODE_FOLDER}")

We will include the inference code as part of the model assets to control the inference process on the SageMaker endpoint. SageMaker will automatically call the `handler()` function for every request to the endpoint.

In [ ]:
%%writefile {ENDPOINT_CODE_FOLDER}/inference.py

import os
import json
import boto3
import requests
import numpy as np
import pandas as pd

from pickle import load
from pathlib import Path


s3 = boto3.resource("s3")


def handler(
    data, 
    context, 
    pipeline_file=Path("/tmp") / "pipeline.pkl", 
    classes_file=Path("/tmp") / "classes.csv"):
    
    """
    This is the entrypoint that will be called by SageMaker when the endpoint
    receives a request. You can see more information at 
    https://github.com/aws/sagemaker-tensorflow-serving-container.
    """
    print("Handling endpoint request")
    
    processed_input, groundtruth = _process_input(data, context, pipeline_file)
    output = _predict(processed_input, context) if processed_input else None
    return _process_output(output, groundtruth, context, classes_file)


def _process_input(data, context, pipeline_file):
    print("Processing input data...")
    
    if context is None:
        # The context will be None when we are testing the code
        # directly from a notebook. In that case, we can use the
        # data directly.
        endpoint_input = data
    elif context.request_content_type in ("application/json", "application/octet-stream"):
        # When the endpoint is running, we will receive a context
        # object. We need to parse the input and turn it into 
        # JSON in that case.
        endpoint_input = json.loads(data.read().decode("utf-8"))

        if endpoint_input is None:
            raise ValueError("There was an error parsing the input request.")
    else:
        raise ValueError(f"Unsupported content type: {context.request_content_type or 'unknown'}")
        
    groundtruth = endpoint_input.pop("species", None)
        
    df = pd.json_normalize(endpoint_input)
    
    try:
        pipeline = _get_pipeline(pipeline_file)
        result = pipeline.transform(df)
    except Exception as e:
        print(f"There was an error transforming the input data. {e}")
        return None, None
    
    return result[0].tolist(), groundtruth


def _predict(instance, context):
    print("Sending input data to model to make a prediction...")
    
    model_input = json.dumps({"instances": [instance]})
    
    if context is None:
        # The context will be None when we are testing the code
        # directly from a notebook. In that case, we want to return
        # a fake prediction back.
        result = {
            "predictions": [
                [0.2, 0.5, 0.3]
            ]
        }
    else:
        # When the endpoint is running, we will receive a context
        # object. In that case we need to send the instance to the
        # model to get a prediction back.
        response = requests.post(context.rest_uri, data=model_input)
        
        if response.status_code != 200:
            raise ValueError(response.content.decode('utf-8'))
            
        result = json.loads(response.content)
    
    print(f"Response: {result}")
    return result


def _process_output(output, groundtruth, context, classes_file):
    print("Processing prediction received from the model...")
    
    if output:
        prediction = np.argmax(output["predictions"][0])
        confidence = output["predictions"][0][prediction]

        result = {
            "prediction": _get_class(prediction, classes_file),
            "confidence": confidence
        }

        if groundtruth:
            result["groundtruth"] = groundtruth
            
    else:
        result = {
            "prediction": None
        }

    print(result)
    
    response_content_type = "application/json" if context is None else context.accept_header
    return json.dumps(result), response_content_type


def _get_pipeline(pipeline_file):
    """
    This function returns the Scikit-Learn pipeline we used to transform the
    dataset.
    """
    
    _download(pipeline_file, os.environ.get("PIPELINE_S3_LOCATION", None))
    return load(open(pipeline_file, 'rb'))


def _get_class(prediction, classes_file):
    """
    This function returns the class name of a given prediction. 
    """
    
    _download(classes_file, os.environ.get("CLASSES_S3_LOCATION", None))
    
    with open(classes_file) as f:
        file = f.readlines()
        
    classes = list(map(lambda x: x.replace("'", ""), file[0].split(',')))
    return classes[prediction]


def _download(file, s3_location):
    """
    This function downloads a file from S3 if it doesn't already exist.
    """
    if file.exists():
        return
        
    s3_parts = s3_location.split('/', 3)
    bucket = s3_parts[2]
    key = s3_parts[3]
    
    s3.Bucket(bucket).download_file(key, str(file))

## Step 3 - Testing the Inference Code

Let's test the inference code locally to ensure it works before deploying it. The `handler()` function is the entry point that will be called by SageMaker whenever the endpoint receives a request.

When testing the inference code, we want to set the `context` to `None` so the function recognizes we are calling it locally.

In [ ]:
from preprocessor import preprocess
from inference import handler, _get_class


samples = [
    {
        "island": "Biscoe",
        "culmen_length_mm": 48.6,
        "culmen_depth_mm": 16.0,
        "flipper_length_mm": 230.0,
        "body_mass_g": 5800.0,
        "species": "Biscoe"
    },
    {
        "island": "Unknown",
        "culmen_length_mm": 48.6,
        "culmen_depth_mm": 16.0,
        "flipper_length_mm": 230.0,
        "body_mass_g": 5800.0,
    }
]

with tempfile.TemporaryDirectory() as directory:
    
    # We want to call the `preprocess()` function to generate the
    # `pipeline.pkl` and `classes.csv` files that we need to use
    # as part of the endpoint inference script.
    preprocess(
        base_directory=directory, 
        data_filepath=DATA_FILEPATH
    )
    
    for sample in samples:
        handler(
            data=sample, 
            context=None,
            pipeline_file=Path(directory) / "pipeline" / "pipeline.pkl",
            classes_file=Path(directory) / "classes" / "classes.csv"
        )
        print()

## Step 4 - Registering the Model

We can now register a new [TensorFlowModel](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-serving-model). We must also ensure SageMaker repackages the model assets to include the `inference.py` file.

SageMaker triggers a repack whenever we specify the `source_dir` attribute. We want that attribute to point to the local folder containing the `inference.py` file. SageMaker will automatically modify the original `model.tar.gz` package to include a `/code` folder containing the file. Since we need access to Scikit-Learn in our script, we can include a `requirements.txt` file in the same `/code` folder, and SageMaker will install everything in it. To repack the model assets, SageMaker will automatically include a new step in the pipeline right before registering the model.

Here is what the new `model.tar.gz` package will look like:

```
model/
    |--[model_version_number]
        |--assets/
        |--variables/
        |--saved_model.pb
code/
    |--inference.py
    |--requirements.txt
```

Let's use a [ModelStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.model_step.ModelStep) to register the model. Notice the following:

* `model_data`: We use the model assets we generated during the Training or Tuning Step. We determined which assets to use back in Session 4 and stored them in the `model_data` variable.
* `source_dir`: This points to the local folder containing the `inference.py` file. SageMaker will trigger a repack to include the `/code` folder in the model assets.
* `env`: Our custom inference code expects an environment variable `S3_LOCATION` to point to the location of the Scikit-Learn pipeline.

SageMaker's default TensorFlow inference container doesn't come with Scikit-Learn installed, so we need to provide a `requirements.txt` file with the libraries we want SageMaker to install in our endpoint.

In [ ]:
%%writefile {ENDPOINT_CODE_FOLDER}/requirements.txt

numpy==1.19.5
pandas==1.2.5
scikit-learn==0.23.2

In [ ]:
model = TensorFlowModel(
    name="penguins",
    model_data=(
        tune_model_step.get_top_model_s3_uri(top_k=0, s3_bucket=pipeline_session.default_bucket())
        if USE_TUNING_STEP
        else train_model_step.properties.ModelArtifacts.S3ModelArtifacts
    ),
    entry_point="inference.py",
    source_dir=str(ENDPOINT_CODE_FOLDER),
    env={
        "PIPELINE_S3_LOCATION": Join(
            on="/",
            values=[
                preprocess_data_step.properties.ProcessingOutputConfig.Outputs["pipeline"].S3Output.S3Uri,
                "pipeline.pkl",
            ]
        ),
        "CLASSES_S3_LOCATION": Join(
            on="/",
            values=[
                preprocess_data_step.properties.ProcessingOutputConfig.Outputs["classes"].S3Output.S3Uri,
                "classes.csv",
            ]
        )
    },
    framework_version="2.6",
    sagemaker_session=pipeline_session,
    role=role,
)

register_model_step = ModelStep(
    name="register",
    display_name="register-model",
    step_args=model.register(
        model_package_group_name=MODEL_PACKAGE_GROUP,
        model_metrics=model_metrics,
        approval_status="Approved",
        content_types=["application/json"],
        response_types=["application/json"],
        inference_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="TENSORFLOW",
        framework_version="2.6",
    )
)

## Step 5 - Creating the Lambda Function

Let's use a [Lambda Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-lambda) to deploy the model automatically.

Let's start by writing the Lambda function to take the model information and create a new hosting endpoint.

In [ ]:
%%writefile {CODE_FOLDER}/lambda.py

import os
import json
import boto3
import time

sagemaker = boto3.client("sagemaker")

def lambda_handler(event, context):
    model_package_arn = event["model_package_arn"]
    endpoint_name = event["endpoint_name"]
    data_capture_percentage = event["data_capture_percentage"]
    data_capture_destination = event["data_capture_destination"]
    role = event["role"]
    
    timestamp = time.strftime("%m%d%H%M%S", time.localtime())
    model_name = f"penguins-model-{timestamp}"
    endpoint_config_name = f"penguins-endpoint-config-{timestamp}"

    sagemaker.create_model(
        ModelName=model_name, 
        ExecutionRoleArn=role, 
        Containers=[{
            "ModelPackageName": model_package_arn
        }] 
    )

    sagemaker.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[
            {
                "ModelName": model_name,
                "InstanceType": "ml.m5.large",
                "InitialVariantWeight": 1,
                "InitialInstanceCount": 1,
                "VariantName": "AllTraffic",
            }
        ],
        DataCaptureConfig={
            "EnableCapture": True,
            "InitialSamplingPercentage": data_capture_percentage,
            "DestinationS3Uri": data_capture_destination,
            "CaptureOptions": [
                {
                    "CaptureMode": "Input"
                },
                {
                    "CaptureMode": "Output"
                },
            ],
            "CaptureContentTypeHeader": {
                "JsonContentTypes": [
                    "application/json",
                    "application/octect-stream"
                ]
            }
        },
    )
    
    response = sagemaker.list_endpoints(NameContains=endpoint_name, MaxResults=1)

    if len(response["Endpoints"]) == 0:
        # If the endpoint doesn't exist, let's create it.
        sagemaker.create_endpoint(
            EndpointName=endpoint_name, 
            EndpointConfigName=endpoint_config_name,
        )
    else:
        # If the endpoint already exist, let's update it with the
        # new configuration.
        sagemaker.update_endpoint(
            EndpointName=endpoint_name, 
            EndpointConfigName=endpoint_config_name,
        )
    
    return {
        "statusCode": 200,
        "body": json.dumps("Endpoint deployed successfully")
    }

We need to ensure our Lambda function has permission to interact with SageMaker, so let's create a new role and then create the lambda function.

In [ ]:
lambda_role_name = "lambda-pipeline-role"

policies = [
    "arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
    "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess"
]

role_arn = None

try:
    response = iam_client.create_role(
        RoleName = lambda_role_name,
        AssumeRolePolicyDocument = json.dumps({
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "lambda.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        }),
        Description="Lambda Pipeline Role"
    )

    role_arn = response["Role"]["Arn"]

    for p in policies:
        iam_client.attach_role_policy(
            RoleName=lambda_role_name,
            PolicyArn=p
        )
except iam_client.exceptions.EntityAlreadyExistsException:
    response = iam_client.get_role(RoleName=lambda_role_name)
    role_arn = response["Role"]["Arn"]

deploy_lambda_fn = Lambda(
    function_name="deploy_fn",
    execution_role_arn=role_arn,
    script=str(CODE_FOLDER / "lambda.py"),
    handler="lambda.lambda_handler",
    timeout=600,
    session=pipeline_session
)

deploy_lambda_fn.upsert()

## Step 6 - Setting up the Lambda Step

Let's define the [LambdaStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.lambda_step.LambdaStep) that will run the function to deploy the model.

We can use [Data Capture](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-data-capture.html) to record the inputs and outputs of the endpoint to use them later for monitoring the model. We'll enable Data Capture using the following settings:

* `data_capture_percentage`: Represents the percentage of information that flows through the endpoint that we want to capture. For this example, we'll set that to 100%.
* `data_capture_destination`: Specifies the S3 location where we want to store the captured data.


In [ ]:
data_capture_percentage = ParameterInteger(
    name="data_capture_percentage",
    default_value=100,
)

data_capture_destination = ParameterString(
    name="data_capture_destination",
    default_value=f"{S3_LOCATION}/monitoring/data-capture",
)

deploy_step = LambdaStep(
    name="deploy",
    lambda_func=deploy_lambda_fn,
    inputs={
        "model_package_arn": register_model_step.properties.ModelPackageArn,
        "endpoint_name": ENDPOINT,
        "data_capture_percentage": data_capture_percentage,
        "data_capture_destination": data_capture_destination,
        "role": role,
    }
)

## Step 7 - Modifying the Condition Step

We need to modify the [Condition Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition) to include the new Deploy Step we just created. If the condition succeeds, we will register and deploy the custom model.

In [ ]:
condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[condition_gte],
    if_steps=[
        register_model_step, deploy_step
    ],
    else_steps=[fail_step], 
)

## Step 8 - Setting up the Pipeline

We can now define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.

In [ ]:
session5_pipeline = Pipeline(
    name="penguins-session5",
    parameters=[
        dataset_location,
        accuracy_threshold,
        data_capture_percentage,
        data_capture_destination,
    ],
    steps=[
        preprocess_data_step, 
        tune_model_step if USE_TUNING_STEP else train_model_step, 
        evaluate_model_step,
        condition_step
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=pipeline_session
)

session5_pipeline.upsert(role_arn=role)

## Step 9 - Testing Custom Endpoint

Let's now test the endpoint we deployed automatically with the pipeline. We will use the function to create a predictor with a JSON encoder and decoder. 

<div class="alert" style="background-color:#0066cc;">Uncomment the <code style="background-color:#0066cc;">%%script</code> cell magic line to execute this cell.</div>

In [ ]:
%%script false --no-raise-error

waiter = sagemaker_client.get_waiter("endpoint_in_service")
waiter.wait(
    EndpointName=ENDPOINT,
    WaiterConfig={
        "Delay": 10,
        "MaxAttempts": 30
    }
)

predictor = Predictor(endpoint_name=ENDPOINT, serializer=JSONSerializer(), deserializer=JSONDeserializer()) 

predictor.predict({
    "island": "Dream",
    "culmen_length_mm": 46.4,
    "culmen_depth_mm": 18.6,
    "flipper_length_mm": 190.0,
    "body_mass_g": 3450.0,
})

Let's now delete the endpoint.

<div class="alert" style="background-color:#0066cc;">Uncomment the <code style="background-color:#0066cc;">%%script</code> cell magic line to execute this cell.</div>

In [ ]:
%%script false --no-raise-error

predictor.delete_endpoint()

# Running the Pipeline

Uncomment the appropriate line to run that specific Session's pipeline. 

<div class="alert" style="background-color:#6e420c; color: #fff"><strong>Note:</strong> 
    Running the pipelines corresponding to Session 5 and Session 6 will deploy the model to an endpoint. Ensure you <strong><a href="https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-delete-resources.html" style="color: #00A7E1">delete this endpoint</a></strong> when you finish using it to prevent extra charges.
</div>

In [ ]:
# session2_pipeline.start()

session3_pipeline.start()
# session4_pipeline.start()

# The following two pipelines deploy the model to an endpoint.
# Ensure you delete the endpoint when you finish using it.
# session5_pipeline.start()
# session6_pipeline.start()